
# 一、实验目的
1. 理解和掌握循环神经网络概念及在深度学习框架中的实现。
2. 掌握使用深度学习框架进行文本生成任务的基本流程：如数据读取、构造网
络、训练和预测等。

# 二、 实验要求
1. 基于 Python 语言和任意一种深度学习框架（实验指导书中使用 Pytorch 框架
进行介绍） ，完成数据读取、网络设计、网络构建、模型训练和模型测试等过
程，最终实现一个可以自动写诗的程序。网络结构设计要有自己的方案，不
能与实验指导书完全相同。
2. 随意给出首句，如给定“湖光秋月两相和”，输出模型续写的诗句。也可以根
据自己的兴趣，进一步实现写藏头诗（不做要求） 。要求输出的诗句尽可能地
满足汉语语法和表达习惯。实验提供预处理后的唐诗数据集，包含 57580 首
唐诗（在课程网站下载） ，也可以使用其他唐诗数据集。
3. 按规定时间在课程网站提交实验报告、代码以及 PPT。


# 三、实验原理


In [241]:
import numpy as np
import torch
import torch.utils
import torch.utils.data

In [242]:

# prepare
'''
    data_loader  数据加载器
    ix2word      序号到词的映射
    word2ix      词到序号的映射
'''

dataset = np.load('./data/tang.npz', allow_pickle=True)
data = dataset['data']
ix2word = dataset['ix2word'].item()
word2ix = dataset['word2ix'].item()
data = torch.from_numpy(data)
data_loader = torch.utils.data.DataLoader(
    data,
    batch_size=16,
    shuffle=True,
    num_workers=0,
)

In [243]:
import torch.nn as nn
class Poetry(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(Poetry, self).__init__()
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input, hidden=None):
        embeds = self.embedding(input)
        batch_size, seq_len = input.size()

        if hidden is None:
            h_0 = input.data.new(2, batch_size, self.hidden_dim).fill_(0).float()
            c_0 = input.data.new(2, batch_size, self.hidden_dim).fill_(0).float()
        else:
            h_0, c_0 = hidden

        output, hidden = self.lstm(embeds, (h_0, c_0))
        output = self.fc(output)
        output = output.reshape(batch_size * seq_len, -1)
        return output, hidden

In [244]:
from HyperPara import paraList
# hyper-parameters in paraList
model = Poetry(
    len(word2ix),
    embedding_dim=paraList.embedding_dim,
    hidden_dim=paraList.hidden_dim
)
optimizer = torch.optim.Adam(model.parameters(),lr=paraList.lr)
criterion = nn.CrossEntropyLoss()
loss_meter = 0

from tqdm import tqdm
def train(model, data_loader, optimizer, criterion):
    model.train()
    loss_meter = 0

    loop = tqdm(data_loader, total=len(data_loader))  # 使用 tqdm 创建进度条
    loop.set_description()

    for i, data in enumerate(loop):
        optimizer.zero_grad()
        data = data.long()
        input, target = data[:, :-1], data[:, 1:]
        output, _ = model(input)
        loss = criterion(output, target.reshape(-1))
        loss.backward()
        optimizer.step()
        loss_meter += loss.item()
        loop.set_description(f'Training Epoch [{i+1}/{len(data_loader)}]')
        loop.set_postfix(loss=loss_meter/(i+1))


def generate(model, start_words, ix2words, word2ix, max_length=100):
    model.train()
    result = list(start_words)
    start_words_len = len(start_words)
    input = torch.Tensor([word2ix['<START>']]).view(1, 1).long()
    hidden = None
    model.eval()

    with torch.no_grad():
        for i in range(max_length):
            output, hidden = model(input, hidden)
            if i < start_words_len:
                w = result[i]
                input = torch.Tensor([word2ix[w]]).view(1, 1).long()
            else:
                top_index = output.data.topk(1)[1].item()
                w = ix2words[top_index]
                result.append(w)
                input = torch.Tensor([top_index]).view(1, 1).long()
            if w == '<EOP>':
                del result[-1]
                break

    return ''.join(result)

In [245]:
EPOCH = 10
for epoch in range(EPOCH):
    print('====> Epoch: {}'.format(epoch+1))
    train(model, data_loader, optimizer, criterion)
    print('====> Say: ', end=' ')
    print(generate(model, '苟利国家生死以', ix2word, word2ix))

====> Epoch: 1


Training Epoch [3599/3599]: 100%|██████████| 3599/3599 [19:10<00:00,  3.13it/s, loss=2.5] 


====> Say:  苟利国家生死以，不知何处不相逢。
====> Epoch: 2


Training Epoch [3599/3599]: 100%|██████████| 3599/3599 [18:48<00:00,  3.19it/s, loss=2.19]


====> Say:  苟利国家生死以，不知何事不能知。我今不得不得意，不得不得不可论。我来不得不可见，不知何事不能知。我今不得不得意，不得不得不可论。我来不得不可见，不知何事不能知。我今不得不得意，不得不得不可论。我来不得
====> Epoch: 3


...

====> Say:  苟利国家生死以，不知何事。不知何处，不得一身。一身不得，一身不同。一生不得，一身不同。一身不得，一身不同。一身不得，一身不同。一身不得，一身不同。一身不得，一身不同。一生不得，一身不死。一身不得，不得
====> Epoch: 9


Training Epoch [3599/3599]: 100%|██████████| 3599/3599 [46:21<00:00,  1.29it/s, loss=1.84]    


====> Say:  苟利国家生死以为名，不得一生。不知何处，无以为人。我不见，不得相与。我不见，不得相。不得一，不得已。不见，不得，不得，不得，不得，不得，不得，不得，不得，不得，不得，不得，不得，不得，不得，不得，不得
====> Epoch: 10


Training Epoch [3599/3599]: 100%|██████████| 3599/3599 [20:14<00:00,  2.96it/s, loss=1.82]

====> Say:  苟利国家生死以，不知何事不能。有时不得一日，无事无为名。有时不得意，无事无为情。有时不得意，无事不得知。不知我何为，不得见我心。我心不可见，我心不可论。我心不可见，我心不可求。我心不可见，我心不可求。


In [246]:
import os
save_dir = './saved_models'
    # 如果目录不存在，创建目录
os.makedirs(save_dir, exist_ok=True)
model_save_path = os.path.join(save_dir, 'model.pth')
# 保存模型
torch.save(model.state_dict(), model_save_path)

print(f"model saved to {model_save_path}")

model saved to ./saved_models/model.pth
